In [188]:
import numpy as np

def init_state():
    # 初始化算法参数
    
    n = 4  # 系统状态维度
    m = 2  # 控制输入维度
    
    # 定义初始化K矩阵
    K_i_0 = np.zeros((n, m))
    # 随机初始化状态x
    x_0 = np.random.rand(4, 1)  
    #print(x_0)
    
    # 初始矩阵 P_i^j 
    P_i_j = np.eye(n)  
    return n, m, K_i_0, x_0, P_i_j

In [189]:
# 获得下一个状态的导数x_i_d
def get_x_i_d(x_i, xi_i):
    # 环境初始化参数，不过用不上
    sigma = 1
    theta = 5
    mu = 2
    k_ij = 0.5
    
    # 定义系统的A和B矩阵
    A = np.array([[0, 0, 0, 0], 
                    [-2, -5, -10, 0], 
                    [1, 2, -2, 1], 
                    [1, 0, 0, -2]])  # 系统矩阵 A
    B = np.array([[1, 0],
                    [-1, 0], 
                    [-1, 2], 
                    [1, 1]])  # 系统矩阵 B
    x_i_d = A @ x_i + B @ xi_i
    return x_i_d


In [190]:

# 函数 P_set 用于生成满足条件的P矩阵集合
# def P_set(k):
#     P_matrices = []
#     norm_limit = 10 * (k + 1)
#     for _ in range(5):  # 随机生成一些候选P矩阵
#         P = np.random.rand(4, 4)
#         P = (P + P.T) / 2  # 将P矩阵对称化
#         if np.linalg.norm(P) <= norm_limit:
#             P_matrices.append(P)
#     return P_matrices

# 获得Γ_i 矩阵
def get_Gamma_1(x_i, K_i_j):
    # 计算控制输入 ξ_i和下一个状态的导数x_i_d
    xi_i = K_i_j @ x_i
    x_i_d = get_x_i_d(x_i, xi_i) # 4*1

    # 定义 Ψ_i, Φ_i, 和 Π_i 矩阵
    Psi_i = np.kron(x_i, x_i).T
    print(tyoe(Psi_i))
    Phi_i = np.vstack([np.triu(x_i * delta_t).flatten() for delta_t in time_intervals[1:]])
    Pi_i = np.vstack(xi_data)

    # 计算 Γ_i 矩阵
    Gamma_i = np.hstack((Phi_i, Pi_i))
    
    # 将状态更新至下一个
    x_new = x_i_d
    return Gamma_i, x_new


# 检查矩阵 Γ_i 是否满足秩条件
def check_Gamma(n, m, Gamma_i):
    if np.linalg.matrix_rank(Gamma_i) >= (n * (n + 1) // 2) + n * m:
        print("Gamma_i：{Gamma_i} 满足秩条件。")
    else:
        print("Gamma_i：{Gamma_i} 不满足秩条件。")


def get_Gamma_2(x_data, K_i_j):
    # 定义初始状态 x_i 和初始控制输入 ξ_i
    x_i = np.random.rand(4, 1)   # 随机初始化
    xi_i = K0_i @ x_i

    # 时间间隔，用于环境中的数据采样
    time_intervals = [0, 1, 2, 3, 4]  # 样本时间间隔

    # 数据收集，用于计算 Ψ_i、Φ_i、Π_i 矩阵
    x_data = []
    xi_data = []

    for t in range(1, len(time_intervals)):
        delta_t = time_intervals[t] - time_intervals[t - 1]
        # 每个时间步更新状态
        x_data.append(np.kron(x_i, x_i.T) * delta_t)
        xi_data.append(np.kron(xi_i, x_i.T) * delta_t)

    x_data = np.vstack(x_data)
    xi_data = np.vstack(xi_data)

    # 定义 Ψ_i, Φ_i, 和 Π_i 矩阵
    Psi_i = x_data
    Phi_i = np.vstack([np.triu(x_i * delta_t).flatten() for delta_t in time_intervals[1:]])
    Pi_i = np.vstack(xi_data)


    # 计算 Γ_i 矩阵
    Gamma_i = np.hstack((Phi_i, Pi_i))

    # 检查矩阵 Γ_i 是否满足秩条件
    if np.linalg.matrix_rank(Gamma_i) >= (4 * (4 + 1) // 2) + 4 * 4:
        print("Gamma_i 满足秩条件。")
    return Gamma_i, x_new



In [191]:
# 定义计算 Hj 和 Kj 的辅助函数,对应公式（10）
def compute_H_K(Gamma_i, P_i, Psi_i):
    # 根据 vec(P_i) 计算 H_i^j 和 K_i^j
    H_K_vec = np.linalg.inv(Gamma_i.T @ Gamma_i) @ Gamma_i.T @ Psi_i @ P_i.flatten()
    H_i_j_vecs = H_K_vec[:4 * (4 + 1) // 2]
    K_i_j_vecs = H_K_vec[4 * (4 + 1) // 2:]

    # 重塑为 H_i^j 和 K_i^j 矩阵
    H_i_j = H_i_j_vecs.reshape(4, 4)
    K_i_j = K_i_j_vecs.reshape(4, 4)
    return H_i_j, K_i_j



In [192]:
# 主循环进行算法迭代
def solve_K(max_iters=1000):
    n, m, K_i_j, x_i, P_i_j = init_state()
    # 初始化 P_i, K_i_j_prev,和迭代计数器 j,k
    P_i_j = np.eye(4)
    P_i_j_0 = P_i_j
    K_i_j_prev = K_i_j
    j = 0
    k = 1

    # 控制算法的参数初始化
    epsilon = 1
    Q = np.eye(4)
    delta_j = lambda j: 1 / (j + 1)

    while j <= max_iters:
        # 获得Gamma_i并更新状态x_i
        Gamma_i, x_i = get_Gamma_1(x_i, K_i_j)

        # 计算 Hj 和 Kj
        H_i_j, K_i_j = compute_H_K(Gamma_i, P_i, Psi_i)

        # 更新 P_i_j
        P_i_j_next = update_P(P_i_j, H_i_j, Q, K_i_j, K_i_j_prev, delta_j)

        # 限制 P 矩阵的范数
        if np.linalg.norm(P_i_j_next, ord='fro') > 10 * (k + 1):
            P_i_j = P_i_j_0
            k = k + 1
        # 判断收敛条件
        elif np.linalg.norm(P_i_j_next - P_i_j, ord='fro') < epsilon:
            print(f"Converged at iteration {j}")
            return K_i_j
        # 否则继续迭代
        else :
            P_i_j = P_i_j_next
        K_i_j_prev = K_i_j
        j += 1
    
    print("Maximum iterations reached without convergence.")
    return K_i_j



In [193]:

K_i_final = solve_K()
print("最终反馈增益矩阵 K:", K_i_final)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 2)